## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-28-04-16-14 +0000,nypost,"Biden, 83, jokes about age, brags about his bo...",https://nypost.com/2026/02/27/us-news/biden-83...
1,2026-02-28-04-07-00 +0000,wsj,Bill Clinton said he had no idea of the crimes...,https://www.wsj.com/politics/house-republicans...
2,2026-02-28-04-05-26 +0000,cbc,"Bolivia cargo plane crash kills at least 15, s...",https://www.cbc.ca/news/world/bolivia-plane-cr...
3,2026-02-28-04-05-12 +0000,bbc,At least 11 killed after military plane carryi...,https://www.bbc.com/news/articles/c0rjpxxpp49o...
4,2026-02-28-04-00-00 +0000,wsj,The Unlikely Coalition of Iranian Students Mob...,https://www.wsj.com/world/middle-east/the-unli...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2557/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
54,trump,30
15,he,18
58,ai,17
347,warner,17
348,bros,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
82,2026-02-27-23-01-00 +0000,wsj,Trump ends government use of Anthropic’s AI mo...,https://www.wsj.com/politics/policy/trump-will...,169
196,2026-02-27-18-25-13 +0000,nypost,Trump makes sobering admission about Iran talk...,https://nypost.com/2026/02/27/us-news/trump-no...,107
1,2026-02-28-04-07-00 +0000,wsj,Bill Clinton said he had no idea of the crimes...,https://www.wsj.com/politics/house-republicans...,107
200,2026-02-27-18-03-16 +0000,nypost,Forcing Bill Clinton to testify will backfire ...,https://nypost.com/2026/02/27/us-news/forcing-...,93
122,2026-02-27-21-50-27 +0000,nypost,Warner Bros. Discovery CEO David Zaslav calls ...,https://nypost.com/2026/02/27/business/warner-...,88


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
82,169,2026-02-27-23-01-00 +0000,wsj,Trump ends government use of Anthropic’s AI mo...,https://www.wsj.com/politics/policy/trump-will...
122,88,2026-02-27-21-50-27 +0000,nypost,Warner Bros. Discovery CEO David Zaslav calls ...,https://nypost.com/2026/02/27/business/warner-...
282,68,2026-02-27-12-00-00 +0000,wsj,Hybrid cyberattacks have become more common. “...,https://www.wsj.com/articles/new-york-state-el...
261,57,2026-02-27-13-19-18 +0000,bbc,Pakistan defence minister says country in 'ope...,https://www.bbc.com/news/articles/cvgjd7pv4y4o...
163,51,2026-02-27-19-54-00 +0000,wsj,The Pentagon said it shot down a “seemingly th...,https://www.wsj.com/politics/national-security...
278,50,2026-02-27-12-02-50 +0000,nypost,Huckabee tells US embassy staff in Israel to l...,https://nypost.com/2026/02/27/us-news/mike-huc...
1,48,2026-02-28-04-07-00 +0000,wsj,Bill Clinton said he had no idea of the crimes...,https://www.wsj.com/politics/house-republicans...
326,43,2026-02-27-09-47-19 +0000,bbc,Green Party wins Gorton and Denton by-election...,https://www.bbc.com/news/articles/cr453rvy6kvo...
165,39,2026-02-27-19-52-17 +0000,startribune,"Anderson: Growing up in south Minneapolis, fis...",https://www.startribune.com/dennis-anderson-la...
51,37,2026-02-28-00-17-31 +0000,startribune,Charges detail planning of St. Paul anti-ICE c...,https://www.startribune.com/attorney-general-p...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
